In [20]:
import numpy as np
from bitstring import Bits
import pickle
import cv2
import time

In [21]:
def Quantization2d(data_arr, min_q, max_q, scale, offset):
	row, col = data_arr.shape
	data_quantization = np.ones((row, col))

	for i in range(row):
		for j in range(col):
			data_quantization[i][j] = round((data_arr[i][j] / scale) + offset)

			if(data_quantization[i][j] > max_q):
				data_quantization[i][j] = max_q
			elif(data_quantization[i][j] < min_q):
				data_quantization[i][j] = min_q

	return data_quantization

def Quantization3d(data, min_q, max_q, scale, offset):
	row, col, num_filter = data.shape
	data_quantization = np.ones((row, col, num_filter))

	for i in range(num_filter):
		data_quantization[:, :, i] = Quantization2d(data[:, :, i], min_q, max_q, scale, offset)
        
	return data_quantization

def Quantization4d(data, min_q, max_q, scale, offset):
	row, col, num_subfilter, num_filter = data.shape
	data_quantization = np.ones((row, col, num_subfilter, num_filter))

	for i in range(num_filter):
		data_quantization[:, :, :, i] = Quantization3d(data[:, :, :, i], min_q, max_q, scale, offset)

	return data_quantization

def Dequantization2d(data_arr, scale, offset):
	row, col = data_arr.shape
	data_dequantization = np.ones((row, col))

	for i in range(row):
		for j in range(col):
			data_dequantization[i][j] = (data_arr[i][j] - offset) * scale

	return data_dequantization

def	Round_quantization2d(data_arr, min_q, max_q):
	row, col = data_arr.shape
	data_round_quantization = np.ones((row, col))

	for i in range(row):
		for j in range(col):
			data = float(data_arr[i][j])
			data_round_quantization[i][j] = round(data)

			if(data_round_quantization[i][j] > max_q):
				data_round_quantization[i][j] = max_q
			elif(data_round_quantization[i][j] < min_q):
				data_round_quantization[i][j] = min_q

	return data_round_quantization

def	Round_quantization3d(data, min_q, max_q):
	row, col, num_filter = data.shape
	data_round_quantization = np.ones((row, col, num_filter))

	for i in range(num_filter):
		data_round_quantization[:, :, i] = Round_quantization2d(data[:, :, i], min_q, max_q)

	return data_round_quantization

In [22]:
def ReLU(data):
	if(data >= 0):
		return data
	else: 
		return 0
def ReLU_2d(data):
	row, col = data.shape
	data_ret = np.zeros((row, col))

	for i in range(row):
		for j in range(col):
			data_ret[i, j] = ReLU(data[i, j])
	
	return data_ret

def ReLU_3d(data):
	row, col, num_filter = data.shape
	data_ret = np.zeros((row, col, num_filter))

	for i in range(num_filter):
		data_ret[:, :, i] = ReLU_2d(data[:, :, i])

	return data_ret

def Sigmoid(data):
    if(data < -4):
        return 0
    elif((-4 <= data) and (data < 0)):
        return (1/2)*((1+(data/4))**2)
    elif((0 <= data) and (data < 4)):
        return 1-(1/2)*((1-(data/4))**2)
    else:
        return 1

def Sigmoid_2d(data):
	row, col = data.shape
	data_ret = np.zeros((row, col))

	for i in range(row):
		for j in range(col):
			data_ret[i, j] = Sigmoid(data[i, j])
	
	return data_ret

def max_pooling(matrix):
	row, col = matrix.shape

	max_value = 0

	for i in range(row):
		for j in range(col):
			if(matrix[i][j] > max_value):
				max_value = matrix[i][j]
	return max_value

def avg_pooling(matrix):
	row, col = matrix.shape

	sum_value = 0

	for i in range(row):
		for j in range(col):
				sum_value = sum_value + matrix[i][j]

	avg_value = sum_value/(col*row)
	return avg_value

def median_pooling(matrix):
	row, col = matrix.shape

	matrix = np.reshape(matrix, (1, row*col))

	matrix = np.sort(matrix)

	num_element = row*col

	median_index = int(num_element/2)

	return matrix[0, median_index]

def pooling_2d(x_matrix, dim_filter, stride, padding, pooling_option):
	x_row, x_col = x_matrix.shape
	filter_row, filter_col = dim_filter

	y_row = int(((x_row - filter_row + 2*padding)/stride) + 1)
	y_col = int(((x_col - filter_col + 2*padding)/stride) + 1)

	if(padding >= 1):
		temp_matrix = x_matrix
		x_row = x_row + 2*padding
		x_col = x_col + 2*padding
		x_matrix = np.zeros((x_row, x_col))
		x_matrix[0+padding:x_row-padding, 0+padding:x_col-padding] = temp_matrix

	y = np.zeros((y_row, y_col))

	index_row = np.arange(start=0, stop=x_row-filter_row+1, step=stride)
	index_col = np.arange(start=0, stop=x_col-filter_col+1, step=stride)
	for i in range(len(index_row)):
		for j in range(len(index_col)):
			sub_matrix = x_matrix[index_row[i]:index_row[i]+filter_row, index_col[j]:index_col[j]+filter_col]

			if(pooling_option == 0):
				y[i, j] = max_pooling(sub_matrix)
			elif(pooling_option == 1):
				y[i, j] = avg_pooling(sub_matrix)
			else:
				y[i, j] = median_pooling(sub_matrix)
			
	return y

def pooling_3d(x_matrix, dim_filter, stride, padding, pooling_option):
	x_row, x_col, num_filter = x_matrix.shape
	filter_row, filter_col = dim_filter

	y_row = int(((x_row - filter_row + 2*padding)/stride) + 1)
	y_col = int(((x_col - filter_col + 2*padding)/stride) + 1)

	y = np.zeros((y_row, y_col, num_filter))

	for i in range(num_filter):
		if(pooling_option == 0):
			y[:, :, i] = pooling_2d(x_matrix[:, :, i], dim_filter, stride, padding, pooling_option)
		elif(pooling_option == 1):
			y[:, :, i] = pooling_2d(x_matrix[:, :, i], dim_filter, stride, padding, pooling_option)
		else:
			y[:, :, i] = pooling_2d(x_matrix[:, :, i], dim_filter, stride, padding, pooling_option)

	return y

def conv(x_matrix, filter_matrix):
	row, col = x_matrix.shape
	y = np.zeros((row, col))

	for i in range(row):
		for j in range(col):
			y[i, j] = x_matrix[i, j]*filter_matrix[i, j]
	
	return y.sum()
	
def conv2(x_matrix, filter_matrix, stride, padding):
	x_row, x_col = x_matrix.shape
	filter_row, filter_col = filter_matrix.shape 

	y_row = int(((x_row - filter_row + 2*padding)/stride) + 1)
	y_col = int(((x_col - filter_col + 2*padding)/stride) + 1)

	if(padding >= 1):
		temp_matrix = x_matrix
		x_row = x_row + 2*padding
		x_col = x_col + 2*padding
		x_matrix = np.zeros((x_row, x_col))
		x_matrix[0+padding:x_row-padding, 0+padding:x_col-padding] = temp_matrix

	y = np.zeros((y_row, y_col))

	index_row = np.arange(start=0, stop=x_row-filter_row+1, step=stride)
	index_col = np.arange(start=0, stop=x_col-filter_col+1, step=stride)
	for i in range(len(index_row)):
		for j in range(len(index_col)):
			sub_matrix = x_matrix[index_row[i]:index_row[i]+filter_row, index_col[j]:index_col[j]+filter_col]
			y[i, j] = conv(sub_matrix, filter_matrix)
	return y

def conv3(x_matrix, filter_matrix, stride, padding):
	row_filter, col_filter, num_sub_filter, num_filter = filter_matrix.shape 				# row_filter_matrix = col_filter_matrix
	row_x, col_x, num_x = x_matrix.shape 													# row_x = col_x
																							# num_sub_filter = num_x
	y_row = int(((row_x - row_filter + 2*padding)/stride) + 1)
	y_col = int(((col_x - col_filter + 2*padding)/stride) + 1)
	
	data = np.zeros((y_row, y_col, num_filter))
	for i in range(num_filter):
		data_temp = np.zeros((y_row, y_col))

		for j in range(num_sub_filter):
			data_temp = data_temp + conv2(x_matrix[:, :, j], filter_matrix[:, :, j, i], stride, padding)
		
		data[:, :, i] = data_temp

	return data

def add_bias_after_conv(conv_result, bias):
	row, col, num_filter = conv_result.shape

	for i in range(num_filter):
		conv_result[:, :, i] = conv_result[:, :, i] + bias[i, :]

	return conv_result

def Feed_forward_Lenet_5_quantization(image, conv0_filter, bias_conv0, conv1_filter, bias_conv1, W0, W1, W2, B0, B1, B2, scale_calculate_convolution_0, scale_for_pooling0, scale_calculate_convolution_1, scale_for_pooling1, scale_for_W0_x_Flatten, scale_for_W1_x_hd0, scale_for_W2_x_hd1, scale_output, scale_for_B0_conv, scale_for_B1_conv, scale_for_B0_fc, scale_for_B1_fc, scale_for_B2_fc, max_conv0_q, min_conv0_q, max_conv1_q, min_conv1_q, max_pooling0_q, min_pooling0_q, max_pooling1_q, min_pooling1_q, max_hd0_q, min_hd0_q, max_hd1_q, min_hd1_q, max_bf_sigmoid_q, min_bf_sigmoid_q, scale_bf_sigmoid):
	stride = 1
	padding = 0
	stride_pooling = 2
	padding_pooling = 0
	pooling_option = 1

	# Convolution layer 0
	conv0_result = conv3(image, conv0_filter, stride, padding)
	add_bias_after_conv0_bf_relu = Round_quantization3d(add_bias_after_conv(conv0_result*scale_calculate_convolution_0, bias_conv0*scale_for_B0_conv), min_conv0_q, max_conv0_q)
	add_bias_after_conv0 = ReLU_3d(add_bias_after_conv0_bf_relu)
	pooling0 = Round_quantization3d(pooling_3d(add_bias_after_conv0, (2, 2), stride_pooling, padding_pooling, pooling_option)*scale_for_pooling0, min_pooling0_q, max_pooling0_q)

	# Convolution layer 1
	conv1_result = conv3(pooling0, conv1_filter, stride, padding)
	add_bias_after_conv1_bf_relu = Round_quantization3d(add_bias_after_conv(conv1_result*scale_calculate_convolution_1, bias_conv1*scale_for_B1_conv), min_conv1_q, max_conv1_q)
	add_bias_after_conv1 = ReLU_3d(add_bias_after_conv1_bf_relu)
	pooling1 = Round_quantization3d(pooling_3d(add_bias_after_conv1, (2, 2), stride_pooling, padding_pooling, pooling_option)*scale_for_pooling1, min_pooling1_q, max_pooling1_q)
	
	# Fully connected
	flatten = pooling1.reshape(-1, 1)
	print('input\n', flatten)
	hd_layer0_bf_relu = Round_quantization2d(np.dot(W0, flatten)*scale_for_W0_x_Flatten + B0*scale_for_B0_fc, min_hd0_q, max_hd0_q)
	hd_layer0 = ReLU_2d(hd_layer0_bf_relu)
	print('hd0\n', hd_layer0)
	hd_layer1_bf_relu = Round_quantization2d(np.dot(W1, hd_layer0)*scale_for_W1_x_hd0 + B1*scale_for_B1_fc, min_hd1_q, max_hd1_q) 
	hd_layer1 = ReLU_2d(hd_layer1_bf_relu)
	print('hd1\n', hd_layer1)
	output_bf_sigmoid = Round_quantization2d(np.dot(W2, hd_layer1)*scale_for_W2_x_hd1 + B2*scale_for_B2_fc, min_bf_sigmoid_q, max_bf_sigmoid_q)
	output = Sigmoid_2d(Dequantization2d(output_bf_sigmoid, scale_bf_sigmoid, 0))
	print('output\n', output)

	return np.argmax(output), flatten

def Load_Conv_and_W_and_Bias(data_path, load_option, conv_shape):
	temp_data = np.loadtxt(data_path)

	if(load_option == 0): # ================== Conv
		row, col, num_sub_filter, num_filter = conv_shape # row = col
		data = np.zeros((row, col, num_sub_filter, num_filter))

		index = 0 # maximum index = num_sub_filter * num_filter
		for m in range(num_filter):
			for k in range(num_sub_filter):
				data[:, :, k, m] = temp_data[index * row: index * row + 5, :]
				index = index + 1
		return data
	elif(load_option == 1): # ================ W
		return temp_data
	else: # ============================= Bias
		return temp_data.reshape(-1, 1)

In [23]:
def convert_int_to_binary(number, bit_width):
	if((bit_width%4) != 0):
		return str(Bits(int=number, length=bit_width))
	else:
		temp =  str(Bits(int=number, length=bit_width+1))
		return '0b' + temp[-(len(temp)-3):]

def convert_binary_to_hex(number, bit_width):
	if(bit_width <= 4):
		return hex(int(number, 2))
	else:
		number = number[2:len(number)]
		data_ret = ''
		index = np.arange(bit_width, 1, -4)
		for i in range(len(index)):
			if(index[i] >= 4):
				temp = hex(int(number[index[i] - 4 : index[i]], 2))
				data_ret = temp[-1:] + data_ret
			else:
				temp = hex(int(number[0 : index[i]], 2))
				data_ret = temp[-1:] + data_ret
		
		return '0x' + data_ret

def create_txt_file_for_w_verilog(data_matrix, num_mutiplier_parallel, bit_width, file_path):
	verilog_file = open(file_path, mode = 'w')

	num_node_next_layer, num_node_previous_layer = data_matrix.shape

	num_sub_matrix = round(num_node_next_layer/num_mutiplier_parallel)
	row_sub_matrix = num_mutiplier_parallel
	col_sub_matrix = num_node_previous_layer

	for i in range(num_sub_matrix):
		sub_matrix = data_matrix[num_mutiplier_parallel*i:num_mutiplier_parallel*(i+1), :]
		for j in range(col_sub_matrix):
			bin_final = ''
			for k in range(row_sub_matrix):
				dec_data = int(sub_matrix[k][j])
				bin_data_temp = convert_int_to_binary(dec_data, bit_width)
				bin_data = bin_data_temp[-bit_width:]
				bin_final = bin_final + bin_data

			sub_bin_final_0 = convert_binary_to_hex('0b' + bin_final[0:round(len(bin_final)/2)], int(bit_width*row_sub_matrix/2))
			sub_bin_final_1 = convert_binary_to_hex('0b' + bin_final[-round(len(bin_final)/2):], int(bit_width*row_sub_matrix/2))

			if((j == (col_sub_matrix - 1)) and (i == num_sub_matrix - 1)):
				verilog_file.write(sub_bin_final_0[-(len(sub_bin_final_0)-2):] + ' ' + sub_bin_final_1[-(len(sub_bin_final_1)-2):])
			else:
				verilog_file.write(sub_bin_final_0[-(len(sub_bin_final_0)-2):] + ' ' + sub_bin_final_1[-(len(sub_bin_final_1)-2):] + '\n')
	verilog_file.close()

def create_txt_file_for_bias_verilog(data_matrix, num_mutiplier_parallel, bit_width, file_path):
	verilog_file = open(file_path, mode = 'w')

	data_matrix = data_matrix.reshape(-1, 1)

	num_node_next_layer, temp = data_matrix.shape

	num_sub_matrix = round(num_node_next_layer/num_mutiplier_parallel)
	len_sub_matrix = num_mutiplier_parallel
	for i in range(num_sub_matrix):
		sub_matrix = data_matrix[num_mutiplier_parallel*i:num_mutiplier_parallel*(i+1)]
		bin_final = ''
		for j in range(len_sub_matrix):
			dec_data = int(sub_matrix[j])
			bin_data_temp = convert_int_to_binary(dec_data, bit_width)
			bin_data = bin_data_temp[-bit_width:]
			bin_final = bin_final + bin_data
		
		sub_bin_final_0 = convert_binary_to_hex('0b' + bin_final[0:round(len(bin_final)/2)], int(bit_width*len_sub_matrix/2))
		sub_bin_final_1 = convert_binary_to_hex('0b' + bin_final[-round(len(bin_final)/2):], int(bit_width*len_sub_matrix/2))

		if(i == num_sub_matrix - 1):
			verilog_file.write(sub_bin_final_0[-(len(sub_bin_final_0)-2):] + ' ' + sub_bin_final_1[-(len(sub_bin_final_1)-2):])
		else:
			verilog_file.write(sub_bin_final_0[-(len(sub_bin_final_0)-2):] + ' ' + sub_bin_final_1[-(len(sub_bin_final_1)-2):] + '\n')
	verilog_file.close()

def create_txt_file_for_input_verilog(input_data, bit_width, file_path):
    verilog_file = open(file_path, mode = 'w')

    input_data = input_data.reshape(-1, 1)

    num_node, temp = input_data.shape

    for i in range(num_node):
        dec_data = int(input_data[i])
        bin_data_temp = convert_int_to_binary(dec_data, bit_width)
        bin_data = bin_data_temp[-bit_width:]
        hex_data = convert_binary_to_hex('0b' + bin_data, bit_width)
        if(i != (num_node - 1)):
            verilog_file.write(str(hex_data[-(len(hex_data)-2):]) + '\n')
        else:
            verilog_file.write(str(hex_data[-(len(hex_data)-2):]))
    verilog_file.close()

In [24]:
max_image = 1
min_image = -1

max_filter_conv0 = 0.64
min_filter_conv0 = -0.64
max_filter_conv1 = 0.94
min_filter_conv1 = -0.94

max_result_conv0 = 2.67
min_result_conv0 = -2.67
max_result_conv1 = 8.88
min_result_conv1 = -8.88

max_bias_conv0 = 0.35
min_bias_conv0 = -0.35
max_bias_conv1 = 0.16
min_bias_conv1 = -0.16

max_pooling0 = 1.75
min_pooling0 = -1.75
max_pooling1 = 2.84
min_pooling1 = -2.84

max_w0 = 1.02
min_w0 = -1.02
max_w1 = 0.78
min_w1 = -0.78
max_w2 = 0.85
min_w2 = -0.85

max_b0 = 0.21
min_b0 = -0.21
max_b1 = 0.19
min_b1 = -0.19
max_b2 = 0.24
min_b2 = -0.24

max_in = 2.84
min_in = -2.84
max_hd0 = 9.6
min_hd0 = -9.6
max_hd1 = 12.8
min_hd1 = -12.8
max_out_bf_act = 23.4
min_out_bf_act = -23.4
max_out = 1
min_out = -1

In [25]:
# ============================================ Bit-width ============================================
bit_width_image = 6
bit_width_filter_conv0 = 6
bit_width_filter_conv1 = 6
bit_width_conv0 = 6
bit_width_conv1 = 6
bit_width_bias_conv0 = 6
bit_width_bias_conv1 = 6
bit_width_pooling0 = 6
bit_width_pooling1 = 6
bit_width_input = bit_width_pooling1

bit_width_w0 = 6
bit_width_w1 = 6
bit_width_w2 = 6

bit_width_b0 = 6
bit_width_b1 = 6
bit_width_b2 = 6

bit_width_h0 = 6
bit_width_h1 = 6
bit_width_bf_sigmoid = 6
bit_width_output = 6

In [26]:
# ============================================ Max-min quantization value ============================================
max_image_q = 2**(bit_width_image-1)-1
min_image_q = -2**(bit_width_image-1)
max_filter_conv0_q = 2**(bit_width_filter_conv0-1)-1
min_filter_conv0_q = -2**(bit_width_filter_conv0-1)
max_filter_conv1_q = 2**(bit_width_filter_conv1-1)-1
min_filter_conv1_q = -2**(bit_width_filter_conv1-1)
max_conv0_q = 2**(bit_width_conv0-1)-1
min_conv0_q = -2**(bit_width_conv0-1)
max_conv1_q = 2**(bit_width_conv1-1)-1
min_conv1_q = -2**(bit_width_conv1-1)
max_bias_conv0_q = 2**(bit_width_bias_conv0-1)-1
min_bias_conv0_q = -2**(bit_width_bias_conv0-1)
max_bias_conv1_q = 2**(bit_width_bias_conv1-1)-1
min_bias_conv1_q = -2**(bit_width_bias_conv1-1)
max_pooling0_q = 2**(bit_width_pooling0-1)-1
min_pooling0_q = -2**(bit_width_pooling0-1)
max_pooling1_q = 2**(bit_width_pooling1-1)-1
min_pooling1_q = -2**(bit_width_pooling1-1)

max_w0_q = 2**(bit_width_w0-1)-1
min_w0_q = -2**(bit_width_w0-1)
max_w1_q = 2**(bit_width_w1-1)-1
min_w1_q = -2**(bit_width_w1-1)
max_w2_q = 2**(bit_width_w2-1)-1
min_w2_q = -2**(bit_width_w2-1)

max_b0_q = 2**(bit_width_b0-1)-1
min_b0_q = -2**(bit_width_b0-1)
max_b1_q = 2**(bit_width_b1-1)-1
min_b1_q = -2**(bit_width_b1-1)
max_b2_q = 2**(bit_width_b2-1)-1
min_b2_q = -2**(bit_width_b2-1)

max_input_q = 2**(bit_width_input-1)-1
min_input_q = -2**(bit_width_input-1)
max_hd0_q = 2**(bit_width_h0-1)-1
min_hd0_q = -2**(bit_width_h0-1)
max_hd1_q = 2**(bit_width_h1-1)-1
min_hd1_q = -2**(bit_width_h1-1)
max_bf_sigmoid_q = 2**(bit_width_bf_sigmoid-1)-1
min_bf_sigmoid_q = -2**(bit_width_bf_sigmoid-1)
max_output_q = 2**(bit_width_output-1)-1
min_output_q = -2**(bit_width_output-1)

In [27]:
# ============================================ Scale and offset for w1 & bias ============================================
# Quantied number = (X/Scale) + Offset
scale_image = (max_image - min_image)/(max_image_q - min_image_q)
scale_filter_conv0 = (max_filter_conv0 - min_filter_conv0)/(max_filter_conv0_q - min_filter_conv0_q)
scale_filter_conv1 = (max_filter_conv1 - min_filter_conv1)/(max_filter_conv1_q - min_filter_conv1_q)
scale_conv0 = (max_result_conv0 - min_result_conv0)/(max_conv0_q - min_conv0_q)
scale_conv1 = (max_result_conv1 - min_result_conv1)/(max_conv1_q - min_conv1_q)
scale_bias_conv0 = (max_bias_conv0 - min_bias_conv0)/(max_bias_conv0_q - min_bias_conv0_q)
scale_bias_conv1 = (max_bias_conv1 - min_bias_conv1)/(max_bias_conv1_q - min_bias_conv1_q)
scale_pooling0 = (max_pooling0 - min_pooling0)/(max_pooling0_q - min_pooling0_q)
scale_pooling1 = (max_pooling1 - min_pooling1)/(max_pooling1_q - min_pooling1_q)

scale_w0 = (max_w0 - min_w0)/(max_w0_q - min_w0_q)
scale_w1 = (max_w1 - min_w1)/(max_w1_q - min_w1_q)
scale_w2 = (max_w2 - min_w2)/(max_w2_q - min_w2_q)

scale_b0 = (max_b0 - min_b0)/(max_b0_q - min_b0_q)
scale_b1 = (max_b1 - min_b1)/(max_b1_q - min_b1_q)
scale_b2 = (max_b2 - min_b2)/(max_b2_q - min_b2_q)

scale_input = (max_in - min_in)/(max_input_q - min_input_q)
scale_hd0 = (max_hd0 - min_hd0)/(max_hd0_q - min_hd0_q)
scale_hd1 = (max_hd1 - min_hd1)/(max_hd1_q - min_hd1_q)
scale_bf_sigmoid = (max_out_bf_act - min_out_bf_act)/(max_bf_sigmoid_q - min_bf_sigmoid_q)
scale_output = (max_out - min_out)/(max_output_q - min_output_q)

offset_image = round((max_image*min_image_q - min_image*max_image_q)/(max_image - min_image))
offset_filter_conv0 = round((max_filter_conv0*min_filter_conv0_q - min_filter_conv0*max_filter_conv0_q)/(max_filter_conv0 - min_filter_conv0))
offset_filter_conv1 = round((max_filter_conv1*min_filter_conv1_q - min_filter_conv1*max_filter_conv1_q)/(max_filter_conv1 - min_filter_conv1))
offset_conv0 = round((max_result_conv0*min_conv0_q - min_result_conv0*max_conv0_q)/(max_result_conv0 - min_result_conv0))
offset_conv1 = round((max_result_conv1*min_conv1_q - min_result_conv1*max_conv1_q)/(max_result_conv1 - min_result_conv1))
offset_bias_conv0 = round((max_bias_conv0*min_bias_conv0_q - min_bias_conv0*max_bias_conv0_q)/(max_bias_conv0 - min_bias_conv0))
offset_bias_conv1 = round((max_bias_conv1*min_bias_conv1_q - min_bias_conv1*max_bias_conv1_q)/(max_bias_conv1 - min_bias_conv1))
offset_pooling0 = round((max_pooling0*min_pooling0_q - min_pooling0*max_pooling0_q)/(max_pooling0 - min_pooling0))
offset_pooling1 = round((max_pooling1*min_pooling1_q - min_pooling1*max_pooling1_q)/(max_pooling1 - min_pooling1))

offset_w0 = round((max_w0*min_w0_q - min_w0*max_w0_q)/(max_w0 - min_w0))
offset_w1 = round((max_w1*min_w1_q - min_w1*max_w1_q)/(max_w1 - min_w1))
offset_w2 = round((max_w2*min_w2_q - min_w2*max_w2_q)/(max_w2 - min_w2))

offset_b0 = round((max_b0*min_b0_q - min_b0*max_b0_q)/(max_b0 - min_b0))
offset_b1 = round((max_b1*min_b1_q - min_b1*max_b1_q)/(max_b1 - min_b1))
offset_b2 = round((max_b2*min_b2_q - min_b2*max_b2_q)/(max_b2 - min_b2))

offset_input = round((max_in*min_input_q - min_in*max_input_q)/(max_in - min_in))
offset_hd0 = round((max_hd0*min_hd0_q - min_hd0*max_hd0_q)/(max_hd0 - min_hd0))
offset_hd1 = round((max_hd1*min_hd1_q - min_hd1*max_hd1_q)/(max_hd1 - min_hd1))
offset_bf_sigmoid = round((max_out_bf_act*min_bf_sigmoid_q - min_out_bf_act*max_bf_sigmoid_q)/(max_out_bf_act - min_out_bf_act))
offset_output = round((max_out*min_output_q - min_out*max_output_q)/(max_out - min_out))

# ============================================ Scale for quantize ============================================
# X * Scale 
scale_calculate_convolution_0 = (scale_image*scale_filter_conv0)/scale_conv0
scale_for_pooling0 = scale_conv0/scale_pooling0
scale_calculate_convolution_1 = (scale_pooling0*scale_filter_conv1)/scale_conv1
scale_for_pooling1 = scale_conv1/scale_pooling1

scale_for_W0_x_Flatten = (scale_input*scale_w0)/scale_hd0
scale_for_W1_x_hd0 = (scale_hd0*scale_w1)/scale_hd1
scale_for_W2_x_hd1 = (scale_hd1*scale_w2)/scale_bf_sigmoid

scale_for_B0_conv = scale_bias_conv0/scale_conv0
scale_for_B1_conv = scale_bias_conv1/scale_conv1

scale_for_B0_fc = scale_b0/scale_hd0
scale_for_B1_fc = scale_b1/scale_hd1
scale_for_B2_fc = scale_b2/scale_bf_sigmoid

In [28]:
num_mutiplier_parallel_for_acc_0 = 120
num_mutiplier_parallel_for_acc_1 = 28
num_mutiplier_parallel_for_acc_2 = 2

Conv0_shape = (5, 5, 1, 6)
Conv1_shape = (5, 5, 6, 16)

image_path_verilog = '../../Weight_and_bias_of_model_verilog/image.txt'
Conv0_path_verilog = '../../Weight_and_bias_of_model_verilog/Conv0.txt'
Bias_conv0_path_verilog = '../../Weight_and_bias_of_model_verilog/Bias_conv0.txt'
Conv1_path_verilog = '../../Weight_and_bias_of_model_verilog/Conv1.txt'
Bias_conv1_path_verilog = '../../Weight_and_bias_of_model_verilog/Bias_conv1.txt'

input_file_path_verilog = '../../Weight_and_bias_of_model_verilog/Input.txt'

W0_path_verilog = '../../Weight_and_bias_of_model_verilog/W1.txt'
W1_path_verilog = '../../Weight_and_bias_of_model_verilog/W2.txt'
W2_path_verilog = '../../Weight_and_bias_of_model_verilog/W3.txt'

B0_path_verilog = '../../Weight_and_bias_of_model_verilog/B1.txt'
B1_path_verilog = '../../Weight_and_bias_of_model_verilog/B2.txt'
B2_path_verilog = '../../Weight_and_bias_of_model_verilog/B3.txt'

Conv0_path = '../../Weight_and_bias_of_model_python/Conv0.txt'
Bias_conv0_path = '../../Weight_and_bias_of_model_python/Bias_conv0.txt'
Conv1_path = '../../Weight_and_bias_of_model_python/Conv1.txt'
Bias_conv1_path = '../../Weight_and_bias_of_model_python/Bias_conv1.txt'

W0_path = '../../Weight_and_bias_of_model_python/W0.txt'
W1_path = '../../Weight_and_bias_of_model_python/W1.txt'
W2_path = '../../Weight_and_bias_of_model_python/W2.txt'

B0_path = '../../Weight_and_bias_of_model_python/B0.txt'
B1_path = '../../Weight_and_bias_of_model_python/B1.txt'
B2_path = '../../Weight_and_bias_of_model_python/B2.txt'

In [29]:
pickle_in = open("../../Dataset/Data_test_10000.pickle","rb")
image_test = pickle.load(pickle_in)
pickle_in = open("../../Dataset/Label_test_10000.pickle","rb")
label_test = pickle.load(pickle_in)

Conv0 = Load_Conv_and_W_and_Bias(Conv0_path, 1, Conv0_shape)
Bias_conv0 = Load_Conv_and_W_and_Bias(Bias_conv0_path, 2, 0)
Conv1 = Load_Conv_and_W_and_Bias(Conv1_path, 1, Conv1_shape)
Bias_conv1 = Load_Conv_and_W_and_Bias(Bias_conv1_path, 2, 0)

W0 = Load_Conv_and_W_and_Bias(W0_path, 1, 0)
W1 = Load_Conv_and_W_and_Bias(W1_path, 1, 0)
W2 = Load_Conv_and_W_and_Bias(W2_path, 1, 0)

B0 = Load_Conv_and_W_and_Bias(B0_path, 2, 0)
B1 = Load_Conv_and_W_and_Bias(B1_path, 2, 0)
B2 = Load_Conv_and_W_and_Bias(B2_path, 2, 0)

In [30]:
Conv0_quant = Quantization2d(Conv0, min_filter_conv0_q, max_filter_conv0_q, scale_filter_conv0, 0)
Bias_conv0_quant = Quantization2d(Bias_conv0, min_bias_conv0_q, max_bias_conv0_q, scale_bias_conv0, 0)
Conv1_quant = Quantization2d(Conv1, min_filter_conv1_q, max_filter_conv1_q, scale_filter_conv1, 0)
Bias_conv1_quant = Quantization2d(Bias_conv1, min_bias_conv1_q, max_bias_conv1_q, scale_bias_conv1, 0)

W0_quant = Quantization2d(W0, min_w0_q, max_w0_q, scale_w0, 0)
W1_quant = Quantization2d(W1, min_w1_q, max_w1_q, scale_w1, 0)
W2_quant = Quantization2d(W2, min_w2_q, max_w2_q, scale_w2, 0)

B0_quant = Quantization2d(B0, min_b0_q, max_b0_q, scale_b0, 0)
B1_quant = Quantization2d(B1, min_b1_q, max_b1_q, scale_b1, 0)
B2_quant = Quantization2d(B2, min_b2_q, max_b2_q, scale_b2, 0)

In [31]:
Conv0 = Load_Conv_and_W_and_Bias(Conv0_path, 0, Conv0_shape)
Bias_conv0 = Load_Conv_and_W_and_Bias(Bias_conv0_path, 2, 0)
Conv1 = Load_Conv_and_W_and_Bias(Conv1_path, 0, Conv1_shape)
Bias_conv1 = Load_Conv_and_W_and_Bias(Bias_conv1_path, 2, 0)

Conv0_quant = Quantization4d(Conv0, min_filter_conv0_q, max_filter_conv0_q, scale_filter_conv0, 0)
Bias_conv0_quant = Quantization2d(Bias_conv0, min_bias_conv0_q, max_bias_conv0_q, scale_bias_conv0, 0)
Conv1_quant = Quantization4d(Conv1, min_filter_conv1_q, max_filter_conv1_q, scale_filter_conv1, 0)
Bias_conv1_quant = Quantization2d(Bias_conv1, min_bias_conv1_q, max_bias_conv1_q, scale_bias_conv1, 0)

In [32]:
quatized_image = Quantization3d(image_test[1], min_input_q, max_input_q, scale_input, 0)

result, input_for_fc = Feed_forward_Lenet_5_quantization(quatized_image, Conv0_quant, Bias_conv0_quant, Conv1_quant, Bias_conv1_quant, W0_quant, W1_quant, W2_quant, B0_quant, B1_quant, B2_quant, scale_calculate_convolution_0, scale_for_pooling0, scale_calculate_convolution_1, scale_for_pooling1, scale_for_W0_x_Flatten, scale_for_W1_x_hd0, scale_for_W2_x_hd1, scale_output, scale_for_B0_conv, scale_for_B1_conv, scale_for_B0_fc, scale_for_B1_fc, scale_for_B2_fc, max_conv0_q, min_conv0_q, max_conv1_q, min_conv1_q, max_pooling0_q, min_pooling0_q, max_pooling1_q, min_pooling1_q, max_hd0_q, min_hd0_q, max_hd1_q, min_hd1_q, max_bf_sigmoid_q, min_bf_sigmoid_q, scale_bf_sigmoid)

input
 [[0.]
 [2.]
 [0.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [3.]
 [1.]
 [0.]
 [5.]
 [0.]
 [0.]
 [1.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [0.]
 [0.]
 [0.]
 [2.]
 [0.]
 [2.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [2.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [0.]
 [2.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [2.]
 [0.]
 [4.]
 [0.]
 [2.]
 [3.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

In [33]:
print(result, label_test[1])

2 2


In [34]:
# =========================== Input ==========================
create_txt_file_for_input_verilog(input_for_fc, bit_width_input, input_file_path_verilog)

create_txt_file_for_input_verilog(quatized_image, bit_width_image, image_path_verilog)

create_txt_file_for_input_verilog(Conv0_quant, bit_width_filter_conv0, Conv0_path_verilog)
create_txt_file_for_input_verilog(Bias_conv0_quant, bit_width_bias_conv0, Bias_conv0_path_verilog)
create_txt_file_for_input_verilog(Conv1_quant, bit_width_filter_conv1, Conv1_path_verilog)
create_txt_file_for_input_verilog(Bias_conv1_quant, bit_width_bias_conv1, Bias_conv1_path_verilog)

In [35]:
create_txt_file_for_w_verilog(W0_quant, num_mutiplier_parallel_for_acc_0, bit_width_w0, W0_path_verilog)
create_txt_file_for_w_verilog(W1_quant, num_mutiplier_parallel_for_acc_1, bit_width_w1, W1_path_verilog)
create_txt_file_for_w_verilog(W2_quant, num_mutiplier_parallel_for_acc_2, bit_width_w2, W2_path_verilog)

In [36]:
create_txt_file_for_bias_verilog(B0_quant, num_mutiplier_parallel_for_acc_0, bit_width_b0, B0_path_verilog)
create_txt_file_for_bias_verilog(B1_quant, num_mutiplier_parallel_for_acc_1, bit_width_b1, B1_path_verilog)
create_txt_file_for_bias_verilog(B2_quant, num_mutiplier_parallel_for_acc_2, bit_width_b2, B2_path_verilog)